<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/04_bodypix_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR BODYPIX - NOTEBOOK 2
================================================================================
Sistema de evaluacion para modelos BodyPix con estructura estandarizada
para comparacion de modelos de segmentacion de personas en fotografia de retrato.

CARACTERISTICAS PRINCIPALES:
- Procesamiento incremental con sistema de checkpoint robusto
- Gestion optima de memoria para Google Colab gratuito
- Guardado automatico despues de cada imagen procesada
- 3 modelos MobileNetV1 con multiplicadores 0.50, 0.75, 1.00
- 4 configuraciones de umbral de segmentacion
- Almacenamiento NPZ por fotografia con mascaras de probabilidad completas
- Segmentacion de partes del cuerpo (caracteristica unica de BodyPix)
- JSON ligero con metricas de rendimiento por umbral
- Visualizaciones estandarizadas

MODELOS DISPONIBLES:
1. MobileNetV1 0.50: Arquitectura ligera optimizada para velocidad
2. MobileNetV1 0.75: Balance entre velocidad y calidad de segmentacion
3. MobileNetV1 1.00: Maxima calidad de segmentacion disponible

CONFIGURACIONES DE UMBRAL:
Las configuraciones evaluan multiples umbrales de segmentacion para analizar
la sensibilidad del modelo a diferentes niveles de confianza:

1. ultra_sensible: [0.3, 0.4, 0.5] - Maxima deteccion de pixels
2. sensibilidad_alta: [0.4, 0.5, 0.6] - Alta sensibilidad
3. sensibilidad_media: [0.5, 0.65, 0.8] - Balance precision-recall
4. baja_sensibilidad: [0.7, 0.8, 0.9] - Alta confianza

ALMACENAMIENTO NPZ (por fotografia):
BodyPix genera mascaras de segmentacion continuas (valores 0-1) que representan
la probabilidad de que cada pixel pertenezca a una persona, y adicionalmente
puede segmentar partes del cuerpo. Se almacena:

Informacion de segmentacion de persona:
- mascara_probabilidad: Array (H, W) con valores 0.0-1.0 continuos
- mascaras_binarias_por_umbral: Dict con mascaras binarias para cada umbral
- dimensiones: Shape de la imagen procesada
- modelo_info: Metadatos del modelo usado

Informacion de partes del cuerpo (CARACTERISTICA UNICA de BodyPix):
- mascara_partes: Array (H, W) con IDs de partes 0-23, o -1 para fondo
- grupo_cara: Mascara binaria agrupada de cara
- grupo_torso: Mascara binaria agrupada de torso
- grupo_brazos: Mascara binaria agrupada de brazos
- grupo_manos: Mascara binaria agrupada de manos
- grupo_piernas: Mascara binaria agrupada de piernas
- grupo_pies: Mascara binaria agrupada de pies

NOTA SOBRE PARTES DEL CUERPO:
La segmentacion de partes del cuerpo es una caracteristica UNICA de BodyPix
que ningun otro modelo del TFM proporciona. Se almacena de forma LIGERA:
- La mascara de partes es int8 (mismo tamaño que la imagen)
- Se pre-calculan 6 grupos semanticos para fotografia de retrato
- Incremento de tamaño: ~7% adicional en NPZ comprimido
- Valor para el TFM: Permite analisis especificos de composicion de retratos

DIFERENCIAS CON OTROS MODELOS:
A diferencia de modelos de deteccion de instancias (YOLO, Mask2Former) o
generacion automatica (SAM2), BodyPix es un modelo de segmentacion semantica
que genera una unica mascara de probabilidad por imagen. No detecta instancias
individuales ni proporciona bounding boxes nativamente.

ESTRUCTURA DE SALIDA:
/TFM/2_Modelos/bodypix/{modelo}/{config}/resultados/
├── json/                           # Metricas por fotografia y umbral
├── mascaras/                       # NPZ con mascaras de probabilidad y partes
├── visualizaciones/                # Visualizaciones por fotografia
└── checkpoint.json                 # Estado del procesamiento

Entrada: Imagenes desde /TFM/0_Imagenes/
Salida: JSON, mascaras NPZ y visualizaciones en estructura organizada

Referencias:
- Papandreou et al. (2018) "PersonLab: Person Pose Estimation and Instance
  Segmentation with a Bottom-Up, Part-Based, Geometric Embedding Model"
- TensorFlow.js BodyPix: https://github.com/tensorflow/tfjs-models/tree/master/body-pix

Autor: Jesus L.
Proyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Octubre 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR BODYPIX - NOTEBOOK 2\n================================================================================\nSistema de evaluacion para modelos BodyPix con estructura estandarizada\npara comparacion de modelos de segmentacion de personas en fotografia de retrato.\n\nCARACTERISTICAS PRINCIPALES:\n- Procesamiento incremental con sistema de checkpoint robusto\n- Gestion optima de memoria para Google Colab gratuito\n- Guardado automatico despues de cada imagen procesada\n- 3 modelos MobileNetV1 con multiplicadores 0.50, 0.75, 1.00\n- 4 configuraciones de umbral de segmentacion\n- Almacenamiento NPZ por fotografia con mascaras de probabilidad completas\n- Segmentacion de partes del cuerpo (caracteristica unica de BodyPix)\n- JSON ligero con metricas de rendimiento por umbral\n- Visualizaciones estandarizadas\n\nMODELOS DISPONIBLES:\n1. MobileNetV1 0.50: Arquitectura ligera optimizada para velocidad\n2.

In [18]:
# =============================================================================
# INSTALACION DE DEPENDENCIAS (EJECUTAR EN COLAB)
# =============================================================================

import subprocess
import sys

print("="*80)
print("INSTALANDO DEPENDENCIAS BODYPIX")
print("="*80)

# Orden correcto de instalacion para evitar conflictos
print("\n[1/5] Actualizando packaging...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "packaging>=24.2.0"])

print("[2/5] Instalando TensorFlow...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tensorflow"])

print("[3/5] Instalando tfjs-graph-converter...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tfjs-graph-converter"])

print("[4/5] Instalando tf-bodypix...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tf-bodypix"])

print("[5/5] Instalando librerias de procesamiento...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "opencv-python", "Pillow", "matplotlib"])

print("\n" + "="*80)
print("DEPENDENCIAS INSTALADAS CORRECTAMENTE")
print("="*80 + "\n")

# =============================================================================
# MONTAR GOOGLE DRIVE
# =============================================================================

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("Google Drive montado correctamente\n")
except:
    IN_COLAB = False
    print("Entorno fuera de Google Colab\n")

INSTALANDO DEPENDENCIAS BODYPIX

[1/5] Actualizando packaging...
[2/5] Instalando TensorFlow...
[3/5] Instalando tfjs-graph-converter...
[4/5] Instalando tf-bodypix...
[5/5] Instalando librerias de procesamiento...

DEPENDENCIAS INSTALADAS CORRECTAMENTE

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente



In [19]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import gc
import time
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

warnings.filterwarnings('ignore')

# Configurar TensorFlow para crecimiento dinamico de memoria GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada: {len(gpus)} dispositivo(s) disponible(s)")
    except RuntimeError as e:
        print(f"Error configurando GPU: {e}")
else:
    print("Ejecutando en CPU")

print("Librerias importadas correctamente\n")

Ejecutando en CPU
Librerias importadas correctamente



In [20]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ModeloBodyPix:
    """Informacion de un modelo BodyPix"""
    nombre: str
    nombre_corto: str
    path: Any  # BodyPixModelPaths enum
    arquitectura: str
    multiplicador: float
    descripcion: str

MODELOS_DISPONIBLES = {
    'mobilenet_v1_050': ModeloBodyPix(
        nombre='MobileNetV1 0.50',
        nombre_corto='mobilenet_v1_050',
        path=BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16,
        arquitectura='MobileNetV1',
        multiplicador=0.50,
        descripcion='Modelo ultraligero optimizado para velocidad'
    ),
    'mobilenet_v1_075': ModeloBodyPix(
        nombre='MobileNetV1 0.75',
        nombre_corto='mobilenet_v1_075',
        path=BodyPixModelPaths.MOBILENET_FLOAT_75_STRIDE_16,
        arquitectura='MobileNetV1',
        multiplicador=0.75,
        descripcion='Balance entre velocidad y calidad de segmentacion'
    ),
    'mobilenet_v1_100': ModeloBodyPix(
        nombre='MobileNetV1 1.00',
        nombre_corto='mobilenet_v1_100',
        path=BodyPixModelPaths.MOBILENET_FLOAT_100_STRIDE_16,
        arquitectura='MobileNetV1',
        multiplicador=1.00,
        descripcion='Maxima calidad de segmentacion disponible'
    )
}

@dataclass
class ConfiguracionUmbrales:
    """
    Configuracion de umbrales de segmentacion para BodyPix.

    Los umbrales controlan que pixels se consideran como persona basandose
    en la probabilidad de segmentacion (valores 0.0 a 1.0).
    """
    nombre: str
    valores: List[float]
    descripcion: str

    def __post_init__(self):
        """Validacion de parametros"""
        if not all(0.0 <= v <= 1.0 for v in self.valores):
            raise ValueError("Los valores de umbral deben estar entre 0.0 y 1.0")

CONFIGURACIONES_UMBRALES = {
    'ultra_sensible': ConfiguracionUmbrales(
        nombre='ultra_sensible',
        valores=[0.3, 0.4, 0.5],
        descripcion='Maxima sensibilidad - Detecta mas pixels como persona'
    ),
    'sensibilidad_alta': ConfiguracionUmbrales(
        nombre='sensibilidad_alta',
        valores=[0.4, 0.5, 0.6],
        descripcion='Alta sensibilidad - Incluye pixels con confianza moderada'
    ),
    'sensibilidad_media': ConfiguracionUmbrales(
        nombre='sensibilidad_media',
        valores=[0.5, 0.65, 0.8],
        descripcion='Balance precision-recall - Configuracion estandar'
    ),
    'baja_sensibilidad': ConfiguracionUmbrales(
        nombre='baja_sensibilidad',
        valores=[0.7, 0.8, 0.9],
        descripcion='Solo pixels de muy alta confianza'
    )
}

@dataclass
class ConfiguracionEvaluacion:
    """Configuracion principal del sistema de evaluacion"""

    # Modelos a evaluar
    modelos_evaluar: List[str] = None  # None = todos los modelos

    # Configuraciones de umbrales a evaluar
    configs_umbrales: List[str] = None  # None = ['sensibilidad_media']

    # Rutas del sistema
    ruta_base: Path = Path("/content/drive/MyDrive/TFM")
    ruta_imagenes: Path = None
    ruta_resultados: Path = None

    # Parametros de procesamiento
    max_dimension: int = 1024  # Dimension maxima para procesamiento
    pausa_entre_imagenes: float = 2.0  # Segundos entre imagenes
    pausa_entre_modelos: float = 5.0   # Segundos entre modelos

    def __post_init__(self):
        """Inicializacion de rutas por defecto"""
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "2_Modelos" / "bodypix"
        if self.modelos_evaluar is None:
            self.modelos_evaluar = list(MODELOS_DISPONIBLES.keys())
        if self.configs_umbrales is None:
            self.configs_umbrales = ['sensibilidad_media']

In [21]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares para procesamiento de imagenes y archivos"""

    @staticmethod
    def cargar_imagen(ruta: Path, max_dim: int = None) -> Tuple[Image.Image, Dict]:
        """
        Carga una imagen con opcion de redimensionamiento.

        Args:
            ruta: Path a la imagen
            max_dim: Dimension maxima (opcional)

        Returns:
            Tupla (imagen, info_dimensiones)
        """
        img = Image.open(ruta).convert("RGB")
        dim_orig = img.size

        redim = False
        if max_dim and max(img.size) > max_dim:
            ratio = max_dim / max(img.size)
            new_size = tuple(int(d * ratio) for d in img.size)
            img = img.resize(new_size, Image.LANCZOS)
            redim = True

        info = {
            'original': {'width': dim_orig[0], 'height': dim_orig[1]},
            'procesada': {
                'width': img.size[0],
                'height': img.size[1],
                'redimensionada': redim
            }
        }
        return img, info

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """Guarda datos en formato JSON"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if tf.config.list_physical_devices('GPU'):
            tf.keras.backend.clear_session()

    @staticmethod
    def obtener_memoria_gpu() -> Dict:
        """Obtiene informacion de memoria GPU disponible"""
        gpus = tf.config.list_physical_devices('GPU')
        if not gpus:
            return {'disponible': False}

        return {
            'disponible': True,
            'dispositivos': len(gpus)
        }

In [22]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """
    Gestiona el estado del procesamiento para permitir reanudacion tras
    interrupciones. Mantiene registro de imagenes completadas por combinacion
    de modelo y configuracion de umbrales.
    """

    def __init__(self, ruta_checkpoint: Path):
        self.ruta_checkpoint = ruta_checkpoint
        self.checkpoint = self._cargar()

    def _cargar(self) -> Dict:
        """Carga checkpoint existente o crea uno nuevo"""
        if self.ruta_checkpoint.exists():
            with open(self.ruta_checkpoint, 'r') as f:
                return json.load(f)
        return {
            'imagenes_completadas': [],
            'fecha_inicio': datetime.now().isoformat()
        }

    def obtener_pendientes(self, todas_imagenes: List[str]) -> set:
        """Retorna conjunto de imagenes pendientes de procesar"""
        completadas = set(self.checkpoint.get('imagenes_completadas', []))
        return set(todas_imagenes) - completadas

    def marcar_completada(self, nombre_imagen: str):
        """Marca una imagen como completada y persiste el checkpoint"""
        if nombre_imagen not in self.checkpoint['imagenes_completadas']:
            self.checkpoint['imagenes_completadas'].append(nombre_imagen)
            self.checkpoint['ultima_actualizacion'] = datetime.now().isoformat()
            Utilidades.guardar_json(self.checkpoint, self.ruta_checkpoint)

In [23]:
# =============================================================================
# GESTOR DE MASCARAS NPZ
# =============================================================================

class GestorMascaras:
    """
    Gestiona el almacenamiento de mascaras en formato NPZ comprimido.

    BodyPix genera una mascara de probabilidad continua (valores 0.0-1.0) y
    opcionalmente una mascara de partes del cuerpo. Se almacenan de forma
    eficiente para analisis posterior sin re-ejecutar el modelo.

    NOTA: La mascara de partes del cuerpo es OPCIONAL y ligera (~mismo tamaño
    que la mascara de persona, ya que se almacena como uint8).
    """

    # Agrupaciones de partes del cuerpo para fotografia de retrato
    # Simplificamos las 24 partes originales en 6 grupos semanticos
    GRUPOS_PARTES_CUERPO = {
        'cara': [0, 1],  # left_face, right_face
        'torso': [12, 13],  # torso_front, torso_back
        'brazos': [2, 3, 4, 5, 6, 7, 8, 9],  # todos los segmentos de brazos
        'manos': [10, 11],  # left_hand, right_hand
        'piernas': [14, 15, 16, 17, 18, 19, 20, 21],  # todos los segmentos de piernas
        'pies': [22, 23]  # left_foot, right_foot
    }

    def __init__(self, directorio_mascaras: Path):
        self.directorio_mascaras = directorio_mascaras
        self.directorio_mascaras.mkdir(parents=True, exist_ok=True)

    def guardar_mascaras(
        self,
        mascara_probabilidad: np.ndarray,
        umbrales: List[float],
        nombre_imagen: str,
        modelo_info: Dict,
        mascara_partes: Optional[np.ndarray] = None
    ) -> str:
        """
        Guarda mascara de probabilidad, binarizaciones por umbral y opcionalmente
        mascara de partes del cuerpo.

        Args:
            mascara_probabilidad: Array (H, W) con valores 0.0-1.0
            umbrales: Lista de umbrales para binarizacion
            nombre_imagen: Nombre del archivo de imagen
            modelo_info: Metadatos del modelo
            mascara_partes: (Opcional) Array (H, W) con IDs de partes 0-23, o -1 para fondo

        Returns:
            Nombre del archivo NPZ generado
        """
        nombre_base = Path(nombre_imagen).stem
        archivo_npz = self.directorio_mascaras / f"{nombre_base}.npz"

        # Preparar mascaras binarias para cada umbral
        mascaras_binarias = {}
        for umbral in umbrales:
            key = f'umbral_{umbral}'
            mascaras_binarias[key] = (mascara_probabilidad >= umbral).astype(np.uint8)

        # Datos base para guardar
        datos_npz = {
            # Mascara de probabilidad original (informacion continua unica de BodyPix)
            'mascara_probabilidad': mascara_probabilidad.astype(np.float32),
            # Mascaras binarias por umbral (pre-calculadas para analisis rapido)
            **mascaras_binarias,
            # Metadatos
            'dimensiones': np.array(mascara_probabilidad.shape, dtype=np.int32),
            'umbrales': np.array(umbrales, dtype=np.float32),
            'modelo_nombre': modelo_info['nombre_corto']
        }

        # Añadir mascara de partes del cuerpo si esta disponible
        if mascara_partes is not None:
            # Almacenar la mascara de partes completa (ligera: uint8)
            # Valores: 0-23 para partes del cuerpo, -1 para fondo
            datos_npz['mascara_partes'] = mascara_partes.astype(np.int8)

            # Pre-calcular mascaras agrupadas para analisis rapido
            # Esto evita tener que procesar las 24 partes en analisis posterior
            for nombre_grupo, ids_partes in self.GRUPOS_PARTES_CUERPO.items():
                mascara_grupo = np.isin(mascara_partes, ids_partes).astype(np.uint8)
                datos_npz[f'grupo_{nombre_grupo}'] = mascara_grupo

        # Guardar en formato NPZ comprimido
        np.savez_compressed(archivo_npz, **datos_npz)

        return archivo_npz.name

In [24]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """Genera visualizaciones estandarizadas de resultados de segmentacion"""

    @staticmethod
    def generar_visualizacion(
        imagen: np.ndarray,
        mascara_probabilidad: np.ndarray,
        umbral_vis: float,
        ruta_salida: Path,
        titulo: str
    ):
        """
        Genera visualizacion de 2 paneles: imagen original y segmentacion.

        Args:
            imagen: Array RGB de la imagen original
            mascara_probabilidad: Mascara de probabilidad (0.0-1.0)
            umbral_vis: Umbral usado para la visualizacion
            ruta_salida: Path donde guardar la visualizacion
            titulo: Titulo para la figura
        """
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)

        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        # Panel 1: Imagen original
        axes[0].imshow(imagen)
        axes[0].set_title('Imagen Original', fontsize=12, fontweight='bold')
        axes[0].axis('off')

        # Panel 2: Segmentacion con overlay
        axes[1].imshow(imagen)

        # Aplicar umbral para visualizacion
        mascara_binaria = mascara_probabilidad >= umbral_vis

        # Crear overlay verde semi-transparente para persona
        overlay = np.zeros_like(imagen)
        overlay[mascara_binaria] = [0, 255, 0]

        axes[1].imshow(overlay, alpha=0.4)
        axes[1].set_title(
            f'Segmentacion (umbral={umbral_vis})',
            fontsize=12,
            fontweight='bold'
        )
        axes[1].axis('off')

        # Calcular y mostrar estadisticas
        area_persona = mascara_binaria.sum()
        area_total = mascara_binaria.size
        porcentaje = (area_persona / area_total) * 100

        info_text = f"Area persona: {porcentaje:.1f}%\n"
        info_text += f"Pixels: {area_persona:,} / {area_total:,}"

        axes[1].text(
            0.02, 0.98, info_text,
            transform=axes[1].transAxes,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=10,
            family='monospace'
        )

        plt.suptitle(titulo, fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=100, bbox_inches='tight')
        plt.close()


In [25]:
# =============================================================================
# PROCESADOR BODYPIX
# =============================================================================

class ProcesadorBodyPix:
    """
    Procesador principal para modelos BodyPix.

    Gestiona la carga del modelo, inferencia y procesamiento de resultados
    para generar mascaras de segmentacion con diferentes umbrales.
    """

    def __init__(self, modelo_info: ModeloBodyPix, config: ConfiguracionEvaluacion):
        self.modelo_info = modelo_info
        self.config = config
        self.model = None

    def cargar_modelo(self):
        """Descarga y carga el modelo BodyPix"""
        print(f"  Cargando modelo {self.modelo_info.nombre}...")
        model_path = download_model(self.modelo_info.path)
        self.model = load_model(model_path)
        print(f"  Modelo cargado correctamente")

    def procesar_imagen(
        self,
        img_path: Path,
        umbrales: List[float],
        ruta_resultados: Path
    ) -> Dict:
        """
        Procesa una imagen con el modelo BodyPix.

        Args:
            img_path: Path a la imagen
            umbrales: Lista de umbrales para evaluacion
            ruta_resultados: Directorio base para resultados

        Returns:
            Diccionario con resultados estructurados y metadatos
        """
        # Cargar imagen
        img, info_img = Utilidades.cargar_imagen(img_path, self.config.max_dimension)
        img_array = np.array(img)

        # Inferencia con BodyPix
        tiempo_inicio = time.time()
        result = self.model.predict_single(img_array)
        tiempo_inferencia_ms = (time.time() - tiempo_inicio) * 1000

        # Obtener mascara de probabilidad base
        # BodyPix genera valores 0-255, normalizar a 0.0-1.0
        mascara_prob_255 = result.get_mask(threshold=0.5).numpy()

        # Asegurar que sea 2D (H, W)
        if mascara_prob_255.ndim == 3:
            # Si tiene canal adicional, tomar solo el primer canal
            mascara_prob_255 = mascara_prob_255[:, :, 0]

        mascara_probabilidad = mascara_prob_255.astype(np.float32) / 255.0

        # Obtener mascara de partes del cuerpo (caracteristica unica de BodyPix)
        # Esto es ligero: solo añade un array int8 del mismo tamaño que la imagen
        mascara_partes = None
        try:
            mascara_partes_raw = result.get_part_mask(img_array)
            # Convertir a array numpy con valores -1 (fondo) o 0-23 (partes)
            mascara_partes = mascara_partes_raw.astype(np.int8)
        except Exception as e:
            # Si falla (modelo sin soporte de partes), continuar sin ellas
            print(f"    Advertencia: No se pudieron obtener partes del cuerpo: {e}")

        # Guardar mascaras en NPZ
        gestor_mascaras = GestorMascaras(ruta_resultados / "mascaras")
        archivo_npz = gestor_mascaras.guardar_mascaras(
            mascara_probabilidad,
            umbrales,
            img_path.name,
            {'nombre_corto': self.modelo_info.nombre_corto},
            mascara_partes=mascara_partes
        )

        # Procesar cada umbral
        detecciones_por_umbral = {}

        for umbral in umbrales:
            # Aplicar umbral
            mascara_binaria = (mascara_probabilidad >= umbral).astype(np.uint8)

            # Calcular metricas
            area_persona = int(mascara_binaria.sum())
            area_total = mascara_binaria.size
            porcentaje_imagen = (area_persona / area_total) * 100

            # Heuristica simple: considerar persona detectada si >= 5% de imagen
            persona_detectada = porcentaje_imagen >= 5.0

            detecciones_por_umbral[f'umbral_{umbral}'] = {
                'umbral_usado': umbral,
                'persona_detectada': persona_detectada,
                'area_pixels': area_persona,
                'area_total_pixels': area_total,
                'porcentaje_imagen': round(porcentaje_imagen, 2),
                'probabilidad_media_region': float(
                    mascara_probabilidad[mascara_binaria > 0].mean()
                    if area_persona > 0 else 0.0
                )
            }

        # Calcular estadisticas de partes del cuerpo si estan disponibles
        info_partes_cuerpo = None
        if mascara_partes is not None:
            info_partes_cuerpo = self._calcular_estadisticas_partes(
                mascara_partes,
                mascara_probabilidad
            )

        # Construir resultado con estructura estandarizada
        resultado = {
            'metadata': {
                'imagen': {
                    'nombre': img_path.name,
                    'ruta': str(img_path),
                    **info_img
                },
                'modelo': {
                    'nombre': self.modelo_info.nombre,
                    'nombre_corto': self.modelo_info.nombre_corto,
                    'arquitectura': self.modelo_info.arquitectura,
                    'multiplicador': self.modelo_info.multiplicador
                },
                'timestamp': datetime.now().isoformat(),
                'tiempo_inferencia_ms': round(tiempo_inferencia_ms, 2)
            },
            'archivos_generados': {
                'mascara_npz': archivo_npz
            },
            'detecciones_por_umbral': detecciones_por_umbral,
            'partes_cuerpo': info_partes_cuerpo  # None si no disponible
        }

        return resultado, img_array, mascara_probabilidad

    def _calcular_estadisticas_partes(
        self,
        mascara_partes: np.ndarray,
        mascara_probabilidad: np.ndarray
    ) -> Dict:
        """
        Calcula estadisticas ligeras sobre las partes del cuerpo detectadas.

        Estas estadisticas son LIGERAS y se calculan rapidamente. El analisis
        detallado de partes se realizara en el Notebook 3 si es necesario.

        Args:
            mascara_partes: Array (H, W) con IDs de partes -1 a 23
            mascara_probabilidad: Array (H, W) con probabilidades 0.0-1.0

        Returns:
            Diccionario con estadisticas basicas por grupo de partes
        """
        # Asegurar que mascara_probabilidad sea 2D
        if mascara_probabilidad.ndim == 3:
            mascara_probabilidad = mascara_probabilidad[:, :, 0]

        # Asegurar que mascara_partes sea 2D
        if mascara_partes.ndim == 3:
            mascara_partes = mascara_partes[:, :, 0]

        area_total = mascara_partes.size
        grupos_detectados = {}

        for nombre_grupo, ids_partes in GestorMascaras.GRUPOS_PARTES_CUERPO.items():
            # Mascara binaria del grupo
            mascara_grupo = np.isin(mascara_partes, ids_partes)
            area_grupo = int(mascara_grupo.sum())

            if area_grupo > 0:
                # Probabilidad media en esta region
                prob_media = float(mascara_probabilidad[mascara_grupo].mean())

                grupos_detectados[nombre_grupo] = {
                    'area_pixels': area_grupo,
                    'porcentaje_imagen': round((area_grupo / area_total) * 100, 2),
                    'probabilidad_media': round(prob_media, 3)
                }

        return {
            'grupos_detectados': grupos_detectados,
            'num_grupos_detectados': len(grupos_detectados),
            'grupos_disponibles': list(GestorMascaras.GRUPOS_PARTES_CUERPO.keys())
        }

    def limpiar_memoria(self):
        """Libera recursos del modelo"""
        del self.model
        self.model = None
        Utilidades.liberar_memoria()

In [26]:
# =============================================================================
# EVALUADOR PRINCIPAL
# =============================================================================

class EvaluadorBodyPix:
    """
    Evaluador principal que coordina el procesamiento de todos los modelos
    y configuraciones de umbrales.
    """

    def __init__(self, config: ConfiguracionEvaluacion):
        self.config = config

    def evaluar_modelo_con_config_umbrales(
        self,
        modelo_key: str,
        config_umbral_nombre: str,
        imagenes: List[Path]
    ) -> Dict:
        """
        Evalua un modelo con una configuracion de umbrales especifica.

        Args:
            modelo_key: Identificador del modelo a evaluar
            config_umbral_nombre: Nombre de la configuracion de umbrales
            imagenes: Lista de paths a imagenes

        Returns:
            Diccionario con estadisticas del procesamiento
        """
        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        umbrales_config = CONFIGURACIONES_UMBRALES[config_umbral_nombre]

        # Ruta de resultados incluye modelo y configuracion
        ruta_resultados = (
            self.config.ruta_resultados /
            modelo_info.nombre_corto /
            config_umbral_nombre /
            "resultados"
        )

        print(f"\n{'='*80}")
        print(f"MODELO: {modelo_info.nombre}")
        print(f"UMBRALES: {umbrales_config.nombre} - {umbrales_config.valores}")
        print(f"{'='*80}")

        # Sistema de checkpoint
        checkpoint = GestorCheckpoint(ruta_resultados / "checkpoint.json")
        nombres_imgs = [img.name for img in imagenes]
        pendientes = checkpoint.obtener_pendientes(nombres_imgs)
        imgs_procesar = [img for img in imagenes if img.name in pendientes]

        print(f"Progreso: {len(imagenes) - len(imgs_procesar)}/{len(imagenes)} completadas")
        print(f"Pendientes: {len(imgs_procesar)}")

        if len(imgs_procesar) == 0:
            print("Todas las imagenes ya procesadas")
            return {'procesadas': 0}

        # Cargar modelo
        procesador = ProcesadorBodyPix(modelo_info, self.config)
        procesador.cargar_modelo()

        # Procesar imagenes pendientes
        tiempo_inicio = time.time()

        for i, img_path in enumerate(imgs_procesar, 1):
            print(f"\n[{i}/{len(imgs_procesar)}] {img_path.name}")

            try:
                # Procesar imagen
                resultado, img_array, mascara_prob = procesador.procesar_imagen(
                    img_path,
                    umbrales_config.valores,
                    ruta_resultados
                )

                # Guardar JSON
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                nombre_json = f"{img_path.stem}_{timestamp}.json"
                ruta_json = ruta_resultados / "json" / nombre_json
                Utilidades.guardar_json(resultado, ruta_json)

                # Generar visualizacion (umbral medio)
                umbral_vis = umbrales_config.valores[len(umbrales_config.valores)//2]
                nombre_vis = f"{img_path.stem}_{timestamp}.png"
                ruta_vis = ruta_resultados / "visualizaciones" / nombre_vis

                GeneradorVisualizaciones.generar_visualizacion(
                    img_array,
                    mascara_prob,
                    umbral_vis,
                    ruta_vis,
                    f"{modelo_info.nombre_corto} - {umbrales_config.nombre}"
                )

                # Marcar como completada
                checkpoint.marcar_completada(img_path.name)

                # Resumen
                umbral_medio_key = f'umbral_{umbral_vis}'
                persona_det = resultado['detecciones_por_umbral'][umbral_medio_key]['persona_detectada']
                print(f"  Completada - Persona detectada: {'Si' if persona_det else 'No'}")

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                import traceback
                traceback.print_exc()

            finally:
                Utilidades.liberar_memoria()
                if i < len(imgs_procesar):
                    time.sleep(self.config.pausa_entre_imagenes)

        tiempo_total = time.time() - tiempo_inicio

        # Limpiar modelo
        procesador.limpiar_memoria()

        print(f"\nConfiguracion completada en {tiempo_total:.1f}s")

        return {
            'procesadas': len(imgs_procesar),
            'tiempo_segundos': round(tiempo_total, 2)
        }

    def ejecutar(self):
        """Ejecuta la evaluacion completa de todos los modelos y configuraciones"""

        print(f"\n{'='*80}")
        print("EVALUADOR BODYPIX - NOTEBOOK 2")
        print(f"{'='*80}")
        print(f"Modelos: {len(self.config.modelos_evaluar)}")
        for modelo_key in self.config.modelos_evaluar:
            modelo = MODELOS_DISPONIBLES[modelo_key]
            print(f"  - {modelo.nombre} (multiplicador {modelo.multiplicador})")

        print(f"\nConfiguraciones de umbrales: {len(self.config.configs_umbrales)}")
        for config_name in self.config.configs_umbrales:
            umbrales = CONFIGURACIONES_UMBRALES[config_name]
            print(f"  - {config_name}: {umbrales.valores}")

        print(f"\nDirectorio de imagenes: {self.config.ruta_imagenes}")
        print(f"Directorio de resultados: {self.config.ruta_resultados}")

        # Cargar imagenes
        imagenes = []
        for extension in ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']:
            imagenes.extend(sorted(self.config.ruta_imagenes.glob(extension)))
        imagenes = sorted(set(imagenes))

        print(f"\nTotal imagenes encontradas: {len(imagenes)}")

        if not imagenes:
            print("ERROR: No se encontraron imagenes en el directorio")
            return

        # Evaluar cada combinacion de modelo + configuracion
        total_combinaciones = (
            len(self.config.modelos_evaluar) *
            len(self.config.configs_umbrales)
        )
        combinacion_actual = 0

        for modelo_key in self.config.modelos_evaluar:
            for config_umbral in self.config.configs_umbrales:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                self.evaluar_modelo_con_config_umbrales(
                    modelo_key,
                    config_umbral,
                    imagenes
                )

                if combinacion_actual < total_combinaciones:
                    print(f"\nPausa entre configuraciones: {self.config.pausa_entre_modelos}s")
                    time.sleep(self.config.pausa_entre_modelos)

        print(f"\n{'='*80}")
        print("EVALUACION COMPLETADA")
        print(f"{'='*80}")
        print(f"Resultados guardados en: {self.config.ruta_resultados}")

In [27]:
# =============================================================================
# EJECUCION PRINCIPAL
# =============================================================================

def main():
    # Configurar evaluacion
    config = ConfiguracionEvaluacion(
        # Seleccionar modelos (None = todos los 3)
        modelos_evaluar=None,

        # Seleccionar configuraciones de umbrales
        configs_umbrales=[
            'ultra_sensible',
            'sensibilidad_alta',
            'sensibilidad_media',
            'baja_sensibilidad'
        ],

        # Parametros de procesamiento
        max_dimension=1024,
        pausa_entre_imagenes=2.0,
        pausa_entre_modelos=5.0
    )

    # Ejecutar evaluacion
    evaluador = EvaluadorBodyPix(config)
    evaluador.ejecutar()

In [28]:
if __name__ == "__main__":
    main()


EVALUADOR BODYPIX - NOTEBOOK 2
Modelos: 3
  - MobileNetV1 0.50 (multiplicador 0.5)
  - MobileNetV1 0.75 (multiplicador 0.75)
  - MobileNetV1 1.00 (multiplicador 1.0)

Configuraciones de umbrales: 4
  - ultra_sensible: [0.3, 0.4, 0.5]
  - sensibilidad_alta: [0.4, 0.5, 0.6]
  - sensibilidad_media: [0.5, 0.65, 0.8]
  - baja_sensibilidad: [0.7, 0.8, 0.9]

Directorio de imagenes: /content/drive/MyDrive/TFM/0_Imagenes
Directorio de resultados: /content/drive/MyDrive/TFM/2_Modelos/bodypix

Total imagenes encontradas: 20

################################################################################
COMBINACION 1/12
################################################################################

MODELO: MobileNetV1 0.50
UMBRALES: ultra_sensible - [0.3, 0.4, 0.5]
Progreso: 0/20 completadas
Pendientes: 20
  Cargando modelo MobileNetV1 0.50...
  Modelo cargado correctamente

[1/20] _DSC0023.jpg
  Completada - Persona detectada: No

[2/20] _DSC0036.jpg
  Completada - Persona detectada: No

[3/

Exception ignored in: <function _xla_gc_callback at 0x7a59ebb5d080>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


  Completada - Persona detectada: No


KeyboardInterrupt: 